In [ ]:

import pandas as pd

print("Loading processed data...")
df = pd.read_csv('processed_funnel_data.csv', parse_dates=['timestamp'])


funnel_stages = [
    'landing_page',
    'product_view',
    'add_to_cart',
    'checkout_start',
    'purchase_complete'
]


print("Calculating funnel metrics...")
funnel_counts = (
    df.groupby('event')
    .agg(users=('user_id', 'nunique'))
    .reindex(funnel_stages)
    .reset_index()
)

funnel_counts['conversion_rate'] = (
    funnel_counts['users'] / funnel_counts['users'].iloc[0]
).round(3)


funnel_counts['drop_off_pct'] = (
    (1 - funnel_counts['users'] / funnel_counts['users'].shift(1).fillna(funnel_counts['users'].iloc[0])) * 100
).round(1).fillna(0)


funnel_counts.to_csv('funnel_metrics.csv', index=False)
print("Funnel metrics calculated:")
print(funnel_counts)


overall_conversion = funnel_counts.iloc[-1]['users'] / funnel_counts.iloc[0]['users']
print(f"\nOverall conversion rate: {overall_conversion:.1%}")


max_drop_idx = funnel_counts['drop_off_pct'].idxmax()
max_drop_stage = funnel_counts.loc[max_drop_idx, 'event']
max_drop_pct = funnel_counts.loc[max_drop_idx, 'drop_off_pct']
print(f"Biggest drop-off at: {max_drop_stage} ({max_drop_pct}%)")

Loading processed data...
Calculating funnel metrics...
Funnel metrics calculated:
               event  users  conversion_rate  drop_off_pct
0       landing_page   5000            1.000           0.0
1       product_view   3321            0.664          33.6
2        add_to_cart   1819            0.364          45.2
3     checkout_start    765            0.153          57.9
4  purchase_complete    590            0.118          22.9

Overall conversion rate: 11.8%
Biggest drop-off at: checkout_start (57.9%)
